In [5]:
from azureml.core import Workspace, Datastore, Experiment, Dataset
from azureml.core.compute import AmlCompute
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData, Pipeline
from azureml.pipeline.core.module import Module
from azureml.pipeline.steps import ModuleStep

ws = Workspace.get(name='jietong-test-westeurope', subscription_id='f8f8b81c-3ebd-427c-b3b7-ebb226c4eea2',
                   resource_group='studio-v2-test')

In [7]:
train_module = Module.get(ws, name='microsoft.com/aml/samples://Train')
score_module = Module.get(ws, name='microsoft.com/aml/samples://Score')
eval_module = Module.get(ws, name='microsoft.com/aml/samples://Evaluate')

train_data = Dataset.get_by_name(ws, 'training_data')
test_data = Dataset.get_by_name(ws, 'test_data')

In [8]:
# PipelineData to represent the connection (data passing) between nodes
# and outputs
ds = ws.get_default_datastore()
trained_model = PipelineData('trained_model', datastore=ds)
scoring_result = PipelineData('scoring_result', datastore=ds)
evaluate_result = PipelineData('evaluate_result', datastore=ds)

# Compute target for the modules to use
compute = AmlCompute(ws, name="cpu-cluster")

# ModuleStep to invoke module
train = ModuleStep(
    train_module,
    inputs_map={'training_data': train_data},
    outputs_map={'model_output': trained_model},
    compute_target=compute,
    params={'Arguments': 'USE_STRUCTURED_ARGUMENTS', 'max_epochs': 5, 'learning_rate': 0.02},
)

score = ModuleStep(
    score_module,
    inputs_map={'model_input': trained_model, 'test_data': test_data},
    outputs_map={'score_output': scoring_result},
    compute_target=compute
)

eval = ModuleStep(
    eval_module,
    inputs_map={'scoring_result': scoring_result},
    outputs_map={'eval_output': evaluate_result},
    compute_target=compute
)

pipeline = Pipeline(ws, steps=[train, score, eval])
experiment = Experiment(ws, 'sample-pipelines')
run = experiment.submit(pipeline)

run.wait_for_completion()

ValueError: Unexpected input type: <class 'azureml.data.file_dataset.FileDataset'>